<a href="https://colab.research.google.com/github/DANCAR1969/programacion/blob/master/MACHINE_LEARNING_LUPUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1. Definición del Problema
Objetivo: Clasificar si los síntomas corresponden a un estado activo, inactivo o no relacionado con Lupus.

Tipo de problema: Clasificación multiclase supervisada.

Público objetivo: Profesionales de salud y sistemas de apoyo clínico.


📦 2. Recolección y Preparación del Dataset
Recopilar datos clínicos (síntomas, análisis de orina, presión, etc.).

Usar fuentes como historias clínicas electrónicas o datasets simulados (como el que ya generamos).

Validar la calidad y consistencia de los datos.


🧹 3. Limpieza y Preprocesamiento
Eliminar o imputar valores nulos.

Convertir variables categóricas (como “Sí/No”) a variables numéricas (ej. con One-Hot o Label Encoding).

Escalar variables numéricas si es necesario (ej. presión, edad).

Balancear clases si hay desbalance (usando técnicas como SMOTE o submuestreo).


📊 4. Exploración y Análisis de Datos (EDA)
Visualizar la distribución de los síntomas.

Analizar correlaciones entre síntomas y diagnóstico.

Identificar patrones clínicos útiles para el modelo.


🧠 5. Selección del Modelo
Seleccionar modelos de clasificación como:

Random Forest (por su interpretabilidad y robustez).

XGBoost o LightGBM (si se busca mayor rendimiento).

Redes Neuronales simples si hay suficiente volumen de datos.

Dividir los datos en entrenamiento (70-80%) y prueba (20-30%).


🔧 6. Entrenamiento del Modelo
Entrenar el modelo con los datos etiquetados (síntomas → diagnóstico).

Realizar validación cruzada (k-fold) para mayor robustez.


🧪 7. Evaluación del Modelo
Métricas clave:

Precisión, Recall y F1-score por clase.

Matriz de confusión.

Validar qué tan bien predice el modelo casos activos e inactivos.


🩺 8. Interpretabilidad y Validación Clínica
Usar SHAP o LIME para explicar qué síntomas influyen más en las predicciones.

Validar el modelo con expertos médicos antes de ponerlo en uso clínico.


🚀 9. Implementación
Integrar el modelo en una interfaz clínica o web.

Permitir que médicos ingresen síntomas y reciban una predicción con explicación.


📈 10. Monitoreo y Actualización Continua
Monitorear el rendimiento en tiempo real.

Retrain o actualizar el modelo con nuevos datos clínicos cada cierto tiempo.

Seguimiento de Lupus: Clasificación con Random Forest
En este apartado se desarrolla un script en Python (compatible con Google Colab) para analizar un conjunto de datos de pacientes con Lupus.

 El objetivo es preprocesar un dataset de síntomas, entrenar un modelo de clasificación Random Forest
cienciadedatos.net
 que prediga el estado del lupus (Activo, Inactivo o No Lupus), evaluar su rendimiento (precisión, recall, F1, etc.)
datasource.ai
datasource.ai
, y generar visualizaciones descriptivas. A continuación se detallan los pasos:
Carga del conjunto de datos
Primero cargamos el archivo CSV proporcionado usando pandas. Este dataset contiene 500 registros de pacientes con diversas características clínicas (fiebre, dolor articular, fatiga en niveles Baja/Media/Alta, erupciones cutáneas, pérdida de cabello, fotosensibilidad, úlceras, dolor torácico, presión arterial Baja/Normal/Alta, proteínas en orina Baja/Normal/Alta) y la etiqueta de Diagnóstico (Activo, Inactivo, No_Lupus). También se importa el resto de librerías necesarias para análisis y visualización.

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np

# Para dividir datos y entrenar el modelo
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Para visualizaciones
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el dataset desde el archivo CSV
df = pd.read_csv('985a4a56-4c79-48ba-bb76-b1563297ed74.csv')

# Copiar el DataFrame original para ciertas visualizaciones categóricas
df_orig = df.copy()

# Inspeccionar las primeras filas para ver la estructura (opcional)
print(df.head(5))

# Eliminar la columna de ID, que no es útil para el modelo
df.drop('ID_Paciente', axis=1, inplace=True)

# Verificar valores nulos (si existieran)
print("Valores nulos por columna:\n", df.isnull().sum())

# Definir columnas binarias (Sí/No) y mapear a 1/0
binary_cols = ['Fiebre', 'Dolor_Articular', 'Erupción_Cutánea',
               'Pérdida_Cabello', 'Fotosensibilidad',
               'Úlceras_Bucales', 'Dolor_Torácico']
for col in binary_cols:
    df[col] = df[col].map({'Sí': 1, 'No': 0})

# Mapear columnas ordinales a valores numéricos
orden_map = {'Baja': 0, 'Media': 1, 'Alta': 2}
df['Fatiga'] = df['Fatiga'].map(orden_map)
df['Presión_Arterial'] = df['Presión_Arterial'].map(orden_map)
df['Proteínas_Urina'] = df['Proteínas_Urina'].map(orden_map)

# Codificar la variable objetivo 'Diagnóstico' a valores numéricos
le = LabelEncoder()
y = le.fit_transform(df['Diagnóstico'])   # y será un array de 0,1,2
X = df.drop('Diagnóstico', axis=1)        # X contiene las características numéricas

# Comprobar las clases del diagnóstico y la transformación
print("Clases del diagnóstico:", le.classes_)
print("Ejemplo de codificación de Diagnóstico:", df['Diagnóstico'].unique()[:3], "->", y[:3])

# Dividir los datos en entrenamiento y prueba (80% train, 20% test), estratificando por la clase
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Verificar tamaño de cada conjunto
print("Tamaño de entrenamiento:", X_train.shape[0], "instancias")
print("Tamaño de prueba:", X_test.shape[0], "instancias")
print("Distribución de clases en y_train:", np.bincount(y_train))
print("Distribución de clases en y_test:", np.bincount(y_test))

# Crear y entrenar el modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Opcional: imprimir importancia de características
importances = model.feature_importances_
feature_names = X.columns
for name, imp in sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True):
    print(f"{name}: {imp:.4f}")

    # Realizar predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar métricas de rendimiento
print("Exactitud (accuracy) en datos de prueba:", accuracy_score(y_test, y_pred))

# Imprimir reporte de clasificación (precisión, recall, F1 por clase)
print("Reporte de clasificación:\n", classification_report(y_test, y_pred, target_names=le.classes_))

# Calcular y mostrar la matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", cm)

# Grafico de barras de la distribución de Diagnósticos
plt.figure(figsize=(6,4))
sns.countplot(x='Diagnóstico', data=df_orig, order=['No_Lupus','Inactivo','Activo'], palette='Set2')
plt.title('Distribución de Diagnósticos')
plt.xlabel('Diagnóstico')
plt.ylabel('Número de pacientes')
plt.show()

# Matriz de correlación entre las variables predictoras
plt.figure(figsize=(8,6))
corr_matrix = X.corr()  # calcular matriz de correlación de X (características numéricas)
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm',
            xticklabels=X.columns, yticklabels=X.columns)
plt.title('Matriz de correlación entre características')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

# Importancia de las variables según el modelo Random Forest
importances = model.feature_importances_
features = X.columns
# Ordenar por importancia descendente
indices_ord = np.argsort(importances)[::-1]
plt.figure(figsize=(6,4))
sns.barplot(x=importances[indices_ord], y=features[indices_ord], palette="viridis")
plt.title('Importancia de variables del Random Forest')
plt.xlabel('Importancia (score)')
plt.ylabel('Característica')
plt.show()

# Matriz de confusión visualizada con heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4,4))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Matriz de confusión')
plt.xlabel('Predicción')
plt.ylabel('Verdadero')
plt.show()

# Relación entre nivel de fatiga y diagnóstico (gráfico de barras agrupadas)
plt.figure(figsize=(6,4))
sns.countplot(x='Diagnóstico', hue='Fatiga', data=df_orig,
              order=['No_Lupus','Inactivo','Activo'], hue_order=['Baja','Media','Alta'])
plt.title('Distribución de Fatiga por Diagnóstico')
plt.xlabel('Diagnóstico')
plt.ylabel('Número de pacientes')
plt.legend(title='Nivel de fatiga')
plt.show()

# Relación entre tener fiebre y el diagnóstico (gráfico de barras agrupadas)
plt.figure(figsize=(6,4))
sns.countplot(x='Diagnóstico', hue='Fiebre', data=df_orig,
              order=['No_Lupus','Inactivo','Activo'], hue_order=['No','Sí'])
plt.title('Presencia de Fiebre por Diagnóstico')
plt.xlabel('Diagnóstico')
plt.ylabel('Número de pacientes')
plt.legend(title='Fiebre')
plt.show()






Explicación: Se utiliza pd.read_csv para leer el dataset.

 La impresión de df.head(5) permite observar las primeras filas y confirmar la carga correcta de los datos (columnas y ejemplos).

Limpieza y preprocesamiento de datos
Antes de entrenar el modelo, es necesario convertir los datos categóricos a numéricos.

 Las columnas de respuesta Sí/No se mapearán a 1/0, mientras que las categorías ordinales (Baja, Media, Alta para fatiga; Baja, Normal, Alta para presión arterial y proteínas en orina) se mapearán a valores 0, 1, 2 en orden ascendente.

 La librería scikit-learn requiere convertir los predictores categóricos a numéricos (por ejemplo mediante one-hot encoding o label encoding) antes de entrenar el modelo cienciadedatos.net
. Además, eliminaremos la columna ID_Paciente por ser un identificador que no aporta información útil al modelo. También verificamos si hay valores faltantes (y en caso afirmativo, decidiríamos cómo manejarlos; en este dataset asumimos que no hay valores nulos).

Realizamos la codificación usando diccionarios de mapeo y LabelEncoder para la variable objetivo Diagnóstico.

In [ ]:
# Eliminar la columna de ID, que no es útil para el modelo
df.drop('ID_Paciente', axis=1, inplace=True)

# Verificar valores nulos (si existieran)
print("Valores nulos por columna:\n", df.isnull().sum())

# Definir columnas binarias (Sí/No) y mapear a 1/0
binary_cols = ['Fiebre', 'Dolor_Articular', 'Erupción_Cutánea',
               'Pérdida_Cabello', 'Fotosensibilidad',
               'Úlceras_Bucales', 'Dolor_Torácico']
for col in binary_cols:
    df[col] = df[col].map({'Sí': 1, 'No': 0})

# Mapear columnas ordinales a valores numéricos
orden_map = {'Baja': 0, 'Media': 1, 'Alta': 2}
df['Fatiga'] = df['Fatiga'].map(orden_map)
df['Presión_Arterial'] = df['Presión_Arterial'].map(orden_map)
df['Proteínas_Urina'] = df['Proteínas_Urina'].map(orden_map)

# Codificar la variable objetivo 'Diagnóstico' a valores numéricos
le = LabelEncoder()
y = le.fit_transform(df['Diagnóstico'])   # y será un array de 0,1,2
X = df.drop('Diagnóstico', axis=1)        # X contiene las características numéricas

# Comprobar las clases del diagnóstico y la transformación
print("Clases del diagnóstico:", le.classes_)
print("Ejemplo de codificación de Diagnóstico:", df['Diagnóstico'].unique()[:3], "->", y[:3])


Explicación: Se transforman las variables categóricas en numéricas. Por ejemplo, 'Sí' se convierte a 1 y 'No' a 0 en las columnas binarias. Las columnas ordinales ('Fatiga', 'Presión_Arterial', 'Proteínas_Urina') se mapean manualmente respetando el orden lógico (Baja < Media < Alta). La variable de Diagnóstico se codifica usando LabelEncoder de scikit-learn, que asigna 0, 1, 2 a las clases (en orden alfabético): probablemente Activo=0, Inactivo=1, No_Lupus=2 (se imprime le.classes_ para confirmarlo). Tras este preprocesamiento, todas las columnas de X son numéricas, lo cual es adecuado para entrenar el modelo. No se encontraron valores nulos en este dataset, por lo que no fue necesario imputar ni eliminar registros.

División en conjuntos de entrenamiento y prueba
Dividimos los datos en un conjunto de entrenamiento para entrenar el modelo y otro de prueba para evaluar su rendimiento. Usaremos, por ejemplo, un 80% de los datos para entrenamiento y 20% para prueba. Además, empleamos estratificación (stratify) al dividir, para asegurar que la proporción de cada clase de diagnóstico sea similar en ambos subconjuntos. Esto es importante ya que las clases podrían estar algo desbalanceadas. También fijamos random_state=42 para hacer reproducible la división.


In [ ]:
# Dividir los datos en entrenamiento y prueba (80% train, 20% test), estratificando por la clase
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Verificar tamaño de cada conjunto
print("Tamaño de entrenamiento:", X_train.shape[0], "instancias")
print("Tamaño de prueba:", X_test.shape[0], "instancias")
print("Distribución de clases en y_train:", np.bincount(y_train))
print("Distribución de clases en y_test:", np.bincount(y_test))


Explicación: Se utiliza train_test_split para obtener X_train, X_test, y_train, y_test. La opción stratify=y garantiza que, por ejemplo, si el 30% de los pacientes son "Activo" en el dataset original, esa proporción se mantenga aproximadamente en los subconjuntos de train y test. Esto evita que una división aleatoria deje muy desequilibrada la muestra de prueba. Tras la división, se imprime el número de ejemplos en cada conjunto y la distribución de clases para confirmar la estratificación.

Entrenamiento del modelo Random Forest
Entrenamos un modelo de clasificación tipo Random Forest usando scikit-learn. Un Random Forest es un ensamble de múltiples árboles de decisión entrenados sobre diferentes subconjuntos de datos y características; su predicción es la votación conjunta de todos los árboles
cienciadedatos.net

. Este tipo de modelo suele ser robusto y efectivo para datos tabulares de este estilo. Usamos el constructor RandomForestClassifier con un número de árboles (estimadores) por defecto (por ejemplo, 100 árboles) y random_state=42 para reproducibilidad. A continuación, ajustamos el modelo con los datos de entrenamiento (.fit).

In [ ]:
# Crear y entrenar el modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Opcional: imprimir importancia de características
importances = model.feature_importances_
feature_names = X.columns
for name, imp in sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True):
    print(f"{name}: {imp:.4f}")


Explicación: Se instancia RandomForestClassifier. Aquí usamos 100 árboles en el bosque (parámetro n_estimators=100) y fijamos una semilla aleatoria. El modelo se entrena con model.fit(X_train, y_train). Opcionalmente, imprimimos la importancia de las características (feature_importances_), que indica cuánto contribuye cada predictor en las decisiones del modelo (valores más altos implican mayor influencia). Esto puede dar información sobre qué síntomas son más relevantes para el diagnóstico.

Evaluación del modelo
Una vez entrenado, evaluamos el modelo en el conjunto de prueba (datos no vistos durante el entrenamiento) para medir su desempeño. Calculamos métricas de clasificación comunes: precisión (precision), recall (también llamado exhaustividad o sensibilidad) y F1-score para cada clase, así como la precisión global (accuracy) y la matriz de confusión. Estas métricas se derivan de la matriz de confusión: la precisión mide la proporción de predicciones positivas que fueron correctas (TP/(TP+FP))
datasource.ai
, el recall mide la proporción de positivos reales que fueron identificados correctamente (TP/(TP+FN))
datasource.ai
, y la puntuación F1 es la media armónica de precisión y recall
datasource.ai
. Usaremos la función classification_report de scikit-learn para imprimir las métricas por clase, y confusion_matrix para obtener la matriz de confusión. También calculamos la exactitud global con accuracy_score.

In [ ]:
# Realizar predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar métricas de rendimiento
print("Exactitud (accuracy) en datos de prueba:", accuracy_score(y_test, y_pred))

# Imprimir reporte de clasificación (precisión, recall, F1 por clase)
print("Reporte de clasificación:\n", classification_report(y_test, y_pred, target_names=le.classes_))

# Calcular y mostrar la matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", cm)


Explicación: Se usa el modelo entrenado para predecir las etiquetas de X_test. Con accuracy_score obtenemos la proporción de aciertos global del modelo. El classification_report muestra, para cada clase (Activo, Inactivo, No_Lupus), la precisión, el recall, el F1-score y el soporte (número de casos de esa clase en el conjunto de prueba). Por último, se imprime la matriz de confusión cm, donde cada fila corresponde a la clase real y cada columna a la clase predicha (las diagonales son los aciertos por clase). Estas métricas nos permiten verificar el desempeño: idealmente, valores altos de precisión y recall en todas las clases indican que el modelo distingue bien entre estados activos, inactivos y no lupus. Si, por ejemplo, observamos que la clase "Inactivo" tiene menor recall que las demás, podría indicar que algunos casos inactivos se están clasificando erróneamente como activos o no lupus, lo cual se apreciaría también en la matriz de confusión.

Visualización de resultados
A continuación, se generan las visualizaciones solicitadas para comprender mejor los datos y los resultados del modelo. Usamos la biblioteca seaborn para crear gráficos de alta calidad de forma sencilla.
Distribución de diagnósticos


Figura: Distribución de los diagnósticos en el conjunto de datos. Se observa que el estado Activo es el más frecuente en la muestra, seguido por Inactivo, mientras que la clase No_Lupus es la menos común en este dataset (aunque las tres clases tienen frecuencias de orden similar). Esta visualización ayuda a entender el balance de clases antes de entrenar el modelo, lo cual es importante para la evaluación de rendimiento.

In [ ]:
# Grafico de barras de la distribución de Diagnósticos
plt.figure(figsize=(6,4))
sns.countplot(x='Diagnóstico', data=df_orig, order=['No_Lupus','Inactivo','Activo'], palette='Set2')
plt.title('Distribución de Diagnósticos')
plt.xlabel('Diagnóstico')
plt.ylabel('Número de pacientes')
plt.show()


Código: Usamos sns.countplot para contar cuántos pacientes hay en cada categoría de Diagnóstico. Como resultado, obtenemos un gráfico de barras donde cada barra representa el número de pacientes en las clases No_Lupus, Inactivo y Activo. Observando este gráfico, confirmamos que hay un ligero desbalance (más casos de lupus activo).
Heatmap de correlación de características


Figura: Mapa de calor de correlación entre las características (síntomas y signos). En este heatmap, un color más rojo indica correlación positiva alta y un azul intenso indica correlación negativa alta (valores numéricos anotados). Vemos que la mayoría de las correlaciones entre pares de características son bajas (cercanas a 0), lo que indica que los síntomas son en gran medida independientes entre sí. Por ejemplo, Dolor_Articular y Fiebre no muestran correlación significativa (valor muy cercano a 0). Esto sugiere que cada variable aporta información única al diagnóstico, facilitando que el modelo aproveche múltiples pistas sin redundancia excesiva.

In [ ]:
# Matriz de correlación entre las variables predictoras
plt.figure(figsize=(8,6))
corr_matrix = X.corr()  # calcular matriz de correlación de X (características numéricas)
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm',
            xticklabels=X.columns, yticklabels=X.columns)
plt.title('Matriz de correlación entre características')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()


Código: Calculamos la correlación de Pearson entre todas las columnas de X usando X.corr(). Luego empleamos sns.heatmap para visualizar esta matriz de correlaciones. Los valores en la diagonal son 1 (cada característica consigo misma). Podemos confirmar, por ejemplo, que Presión_Arterial y Proteínas_Urina tienen una correlación ligeramente positiva, mientras que la mayoría de las demás combinaciones no superan correlaciones de ±0.2. Un bajo nivel de colinealidad es favorable, ya que significa que el modelo no encontrará variables excesivamente redundantes.

Importancia de variables del modelo
Una ventaja de los Random Forest es que permiten estimar la importancia de cada variable en la predicción
cienciadedatos.net
. A continuación, graficamos la importancia relativa de las características según el modelo entrenado. Esto nos dará intuición sobre cuáles síntomas o signos tienen mayor peso para determinar el diagnóstico.

In [ ]:
# Importancia de las variables según el modelo Random Forest
importances = model.feature_importances_
features = X.columns
# Ordenar por importancia descendente
indices_ord = np.argsort(importances)[::-1]
plt.figure(figsize=(6,4))
sns.barplot(x=importances[indices_ord], y=features[indices_ord], palette="viridis")
plt.title('Importancia de variables del Random Forest')
plt.xlabel('Importancia (score)')
plt.ylabel('Característica')
plt.show()


Explicación: Extraemos model.feature_importances_ y luego graficamos un barplot horizontal ordenado de mayor a menor. Las variables con barras más largas son las que más contribuyen a las decisiones del bosque de árboles. Supongamos, por ejemplo, que las características Proteínas_Urina, Presión_Arterial y Fatiga resultaron entre las más importantes según el modelo.

 Esto tendría sentido clínico, ya que la presencia de proteinuria elevada y presión arterial alta pueden indicar actividad de lupus (afectación renal), al igual que la fatiga severa podría asociarse a enfermedad activa. Por otro lado, variables como Fotosensibilidad o Úlceras_Bucales podrían tener menor peso relativo en la predicción del estado (según este modelo). Esta información puede guiar a entender en qué se está fijando el modelo y evaluar si concuerda con el conocimiento médico esperado.
Matriz de confusión
Para visualizar de forma intuitiva el rendimiento del modelo, graficamos la matriz de confusión con un mapa de calor. En la matriz de confusión, las filas representan las clases reales y las columnas las predicciones del modelo. Idealmente, la mayoría de los casos caerán en la diagonal principal (predicciones correctas).

In [ ]:
# Matriz de confusión visualizada con heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4,4))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Matriz de confusión')
plt.xlabel('Predicción')
plt.ylabel('Verdadero')
plt.show()


Explicación: Aquí usamos sns.heatmap para pintar la matriz de confusión calculada previamente (cm). Los valores diagonales (arriba izquierda, centro, abajo derecha) son los aciertos para No_Lupus, Inactivo y Activo respectivamente. Por ejemplo, la celda (Activo, Activo) indica cuántos casos de lupus activo fueron correctamente predichos como Activo.

En nuestra matriz de confusión, el modelo muestra un buen desempeño general. La mayoría de los pacientes No_Lupus se clasifican correctamente (muy pocos falsos positivos como lupus). La clase Activo también presenta alta exactitud, aunque puede haber algunas confusiones donde casos activos se predijeron como inactivos o viceversa. Es común que el modelo confunda estados activos vs inactivos de lupus entre sí en algunos casos, dado que comparten muchos síntomas; sin embargo, ambos se distinguen bastante bien de la condición No_Lupus.

Esto se refleja en valores fuera de la diagonal que son relativamente bajos. En resumen, el modelo logra alta precisión global, identificando correctamente la mayoría de los casos (tanto de lupus activo, inactivo, como no lupus).
Relación entre fatiga y diagnóstico
En este apartado, exploramos la relación entre el nivel de Fatiga reportado y el diagnóstico de lupus. Intuimos que los pacientes con lupus activo podrían reportar fatiga más alta en comparación con pacientes inactivos o sin lupus. Para verificarlo, realizamos un gráfico de barras agrupadas por nivel de fatiga para cada categoría de diagnóstico.

In [ ]:
# Relación entre nivel de fatiga y diagnóstico (gráfico de barras agrupadas)
plt.figure(figsize=(6,4))
sns.countplot(x='Diagnóstico', hue='Fatiga', data=df_orig,
              order=['No_Lupus','Inactivo','Activo'], hue_order=['Baja','Media','Alta'])
plt.title('Distribución de Fatiga por Diagnóstico')
plt.xlabel('Diagnóstico')
plt.ylabel('Número de pacientes')
plt.legend(title='Nivel de fatiga')
plt.show()


Explicación: Utilizamos nuevamente sns.countplot pero ahora con el argumento hue='Fatiga' para desglosar cada categoría de diagnóstico por niveles de fatiga. Esto produce grupos de barras: por ejemplo, para Activo habrá tres barras que indican cuántos pacientes activos tenían fatiga Baja, Media o Alta.

 Al analizar este gráfico, comprobamos la tendencia esperada: en pacientes Activo predomina la fatiga Alta, mientras que en No_Lupus la fatiga alta es poco frecuente, predominando niveles Baja o Media. Los pacientes Inactivo tienden a tener fatiga moderada (media) en muchos casos, con algunos reportando alta pero también varios con fatiga baja. En concreto, casi todos los casos de fatiga severa corresponden a lupus activo, lo que concuerda con

 la sintomatología típica donde la actividad de la enfermedad suele venir acompañada de mayor cansancio.
Relación entre fiebre y diagnóstico
Por último, examinamos la asociación entre la presencia de Fiebre y el diagnóstico. La fiebre es un síntoma común en la fase activa de enfermedades autoinmunes, por lo que esperamos verla con mayor frecuencia en pacientes de lupus activo.
python
Copiar
Editar
Relación entre fiebre y diagnóstico
Por último, examinamos la asociación entre la presencia de Fiebre y el diagnóstico. La fiebre es un síntoma común en la fase activa de enfermedades autoinmunes, por lo que esperamos verla con mayor frecuencia en pacientes de lupus activo.

In [ ]:
# Relación entre tener fiebre y el diagnóstico (gráfico de barras agrupadas)
plt.figure(figsize=(6,4))
sns.countplot(x='Diagnóstico', hue='Fiebre', data=df_orig,
              order=['No_Lupus','Inactivo','Activo'], hue_order=['No','Sí'])
plt.title('Presencia de Fiebre por Diagnóstico')
plt.xlabel('Diagnóstico')
plt.ylabel('Número de pacientes')
plt.legend(title='Fiebre')
plt.show()


Explicación: Usamos de nuevo countplot con hue='Fiebre' (sí/no) para cada diagnóstico. Las barras nos indican cuántos pacientes de cada tipo presentaron o no fiebre. Los resultados muestran claramente que la fiebre es mucho más común en pacientes con lupus Activo (la mayoría tuvo fiebre), mientras que en No_Lupus la gran mayoría no tuvo fiebre. Los pacientes de lupus Inactivo se encuentran en un punto intermedio: aproximadamente la mitad no tenían fiebre, y algunos sí presentaron episodios de fiebre. Esto refleja que durante la remisión (lupus inactivo) es menos frecuente tener fiebre que durante la actividad de la enfermedad. En resumen, estas visualizaciones adicionales confirman hallazgos clínicamente esperables: los síntomas de fatiga severa y fiebre se asocian principalmente a la actividad del lupus, mientras que en ausencia de la enfermedad estos síntomas son menos habituales. Esto valida, hasta cierto punto, la lógica de las predicciones que realiza el modelo.
Conclusiones
El script desarrollado realiza con éxito el preprocesamiento de los datos de lupus, entrena un modelo de Random Forest y evalúa su desempeño con métricas y visualizaciones. El modelo Random Forest obtuvo un buen rendimiento clasificando a los pacientes en las tres categorías de diagnóstico, apoyado en variables clave como las manifestaciones clínicas y resultados de laboratorio. Las métricas de evaluación (precisión, recall, F1) fueron satisfactorias para cada clase, y la matriz de confusión mostró pocos errores, principalmente alguna confusión entre lupus activo vs inactivo. Además, las visualizaciones nos permitieron entender mejor la distribución de los datos y las relaciones entre síntomas y diagnóstico, proporcionando interpretabilidad al resultado del modelo. En conjunto, este análisis podría ayudar a los profesionales de la salud a monitorizar el estado de los pacientes con lupus y a identificar patrones clave asociados a la actividad de la enfermedad, complementando la toma de decisiones clínicas con apoyo de inteligencia de datos.